In [15]:
# 500개 단어 --> 2000개 단어로
# 길이 제한 100 --> 300 

In [16]:
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 표시 해결
# 한글설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # windows 사용자
# matplotlib.rcParams['font.family'] = 'AppleGothic Gothic' # Mac사용자
matplotlib.rcParams['font.size'] = '10' # 글자크기

In [17]:
# imdb 데이터 불러오기
# 단어 사전 1000개 까지 가져옴.
(train_input, train_target), (test_input, test_target) = keras.datasets.imdb.load_data(
    num_words=2000
)

In [18]:
print(train_input.shape,test_input.shape)

(25000,) (25000,)


In [19]:
train_input, val_input, train_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [20]:
# 시퀀스 패딩 제한=200
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_seq = pad_sequences(train_input, maxlen=300)

In [21]:
train_seq.shape

(20000, 300)

In [22]:
# 검증 세트 - 시퀀스 패딩
val_seq = pad_sequences(val_input, maxlen=300)

In [23]:
model = keras.Sequential()

# 임베딩층 추가
model.add(keras.layers.Embedding(2000,16,input_length = 300))
model.add(keras.layers.SimpleRNN(8))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
rmsprop = keras.optimizers.RMSprop(learning_rate=0.0001)
model.compile(optimizer=rmsprop,loss='binary_crossentropy',metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('simpleRnn_embedding_model.keras')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True )
history = model.fit(train_seq,train_target,batch_size=64,epochs=100,
                     validation_data=(val_seq,val_target), callbacks=[checkpoint_cb,early_stopping_cb])

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.5158 - loss: 0.6951 - val_accuracy: 0.6048 - val_loss: 0.6771
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.6452 - loss: 0.6659 - val_accuracy: 0.7058 - val_loss: 0.6472
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.7244 - loss: 0.6345 - val_accuracy: 0.7474 - val_loss: 0.6176
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.7670 - loss: 0.5996 - val_accuracy: 0.7626 - val_loss: 0.5896
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.7933 - loss: 0.5672 - val_accuracy: 0.7950 - val_loss: 0.5574
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.8049 - loss: 0.5428 - val_accuracy: 0.8034 - val_loss: 0.5328
Epoch 7/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.8177 - loss: 0.5108 - val_accuracy: 0.8168 - val_loss: 0.5036
Epoch 8/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.8335 - loss: 0.4786 -

In [25]:
early_stopping_cb.stopped_epoch

26

In [26]:
model.evaluate(val_seq,val_target)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8519 - loss: 0.3521


[0.3475090563297272, 0.8579999804496765]